In [1]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [ ]:
pip install fasttext

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     |████████████████████████████████| 68 kB 6.3 MB/s 
  Using cached pybind11-2.9.2-py2.py3-none-any.whl (213 kB)


In [ ]:
# import dataclasses
import numpy as np
import pandas as pd
import re


In [4]:
import fasttext
import fasttext.util
ft = fasttext.load_model('/content/drive/MyDrive/DEBI/uottawa/Ds applications/kaggle competition/fasttext/wiki.ar.bin')
print(ft.get_dimension())

# fasttext.util.reduce_model(ft, 100)
# ft.get_dimension()

300


In [ ]:
ft.get_nearest_neighbors('اهلا')

In [ ]:
help(fasttext.FastText)

In [6]:
directory = "/content/drive/MyDrive/DEBI/uottawa/Ds applications/kaggle competition/"
data_frame = pd.read_csv(directory+"araBERT/almost_cleaned_data.csv")
data_frame.head(10)

,tweet,class
0,ما دام ورد ذكر حرف الدال ببدل بدي فانو الجوع ف...,neg
1,حمدا لله على سلامة أبنائنا المجندين المحررين و...,pos
2,إحباط,neg
3,بالفيديو البابا تواضروس عن ماسبيرو حادث مثل أي...,neu
4,في ناس واقفة بشوم عند الشوارع المؤدية للتحرير ...,neg
5,تصبحوا على خير,neu
6,لم يتبقى شيئا بعد,neu
7,بعض الأصدقاء الذين أحترمهم رفضوا فكرتي بفظاظة ...,neu
8,لما الموب بتاعي يبوظ فقط الأيام غير سيباندا غي...,neu
9,يوم عظيم على شعب عظيم بإذن الله حشود المصريين ...,pos


In [7]:

# #============= Read CSV and apply data preperation =============#
# df = pd.read_csv("/content/drive/MyDrive/DEBI/uottawa/Ds applications/kaggle competition/train.csv")

# # clean-up: remove #tags, http links and special symbols
# 
# df.iloc[:,0] = df.iloc[:,0].apply(lambda x: re.sub(r'http\S+', '', x))
# df.iloc[:,0] = df.iloc[:,0].apply(lambda x: re.sub(r'[@|#]\S*', '', x))
# df.iloc[:,0] = df.iloc[:,0].apply(lambda x: re.sub(r'"+', '', x))

# # Remove arabic signs
# df.iloc[:,0] = df.iloc[:,0].apply(lambda x: re.sub(r'([@A-Za-z0-9_ـــــــــــــ]+)|[^\w\s]|#|http\S+', '', x))

# # Remove repeated letters like "الللللللللللللللله" to "الله"
# df.iloc[:,0] = df.iloc[:,0].apply(lambda x: x[0:2] + ''.join([x[i] for i in range(2, len(x)) if x[i]!=x[i-1] or x[i]!=x[i-2]]))

# # Tokenize the sentences using bert tokenizer
general_filter = lambda x: re.sub(r'([@A-Za-z0-9ـــــــــــــ]+)|[^\w\s]|#|http\S+|[\s]{2,}', '', x)
_filter = lambda x: re.sub(r'_', ' ', x)
rem_repeated_letters = lambda x: x[0:2] + ''.join([x[i] for i in range(2, len(x)) if x[i]!=x[i-1] or x[i]!=x[i-2]])
heart_emotion_translate = lambda x: re.sub(r'[♥|❤️|❤|♡]+', 'قلب', x)
happy_emotion_translate = lambda x: re.sub(r'(\^[._]?\^)+', 'سعيد', x)
sad_emotion_translate = lambda x: re.sub(r'(-[._]?-)+', 'حزين', x)

text =""

def pre_process(data_frame):
  data_frame_x = data_frame.copy()
  # data_frame_x.iloc[:,0] = data_frame_x.iloc[:,0].apply(lambda x: x[2:-2])
  # data_frame_x.iloc[:,0] = data_frame_x.iloc[:,0].apply(heart_emotion_translate)
  # data_frame_x.iloc[:,0] = data_frame_x.iloc[:,0].apply(happy_emotion_translate)
  # data_frame_x.iloc[:,0] = data_frame_x.iloc[:,0].apply(sad_emotion_translate)
  # data_frame_x.iloc[:,0] = data_frame_x.iloc[:,0].apply(general_filter)
  # data_frame_x.iloc[:,0] = data_frame_x.iloc[:,0].apply(_filter)
  # data_frame_x.iloc[:,0] = data_frame_x.iloc[:,0].apply(rem_repeated_letters) 
  # data_frame_x.iloc[:,0] = data_frame_x.iloc[:,0].apply(lem_stopwords_tokenize)
  lines=[]
  for i in range(len(data_frame_x.iloc[:,0])):
    line =  f'__label__{data_frame_x.iloc[i,1]} '+data_frame_x.iloc[i,0]
    # print(line)
    lines.append(line)
  text = '\n'.join(lines)
  data_frame_x["fasttext_tokens"] =lines
  return data_frame_x,text

In [8]:
df, text = pre_process(data_frame)


In [9]:
with open(directory+'fasttext/text.txt', 'w') as f:
    f.write(text)

In [10]:
df

,tweet,class,fasttext_tokens
0,ما دام ورد ذكر حرف الدال ببدل بدي فانو الجوع ف...,neg,__label__neg ما دام ورد ذكر حرف الدال ببدل بدي...
1,حمدا لله على سلامة أبنائنا المجندين المحررين و...,pos,__label__pos حمدا لله على سلامة أبنائنا المجند...
2,إحباط,neg,__label__neg إحباط
3,بالفيديو البابا تواضروس عن ماسبيرو حادث مثل أي...,neu,__label__neu بالفيديو البابا تواضروس عن ماسبير...
4,في ناس واقفة بشوم عند الشوارع المؤدية للتحرير ...,neg,__label__neg في ناس واقفة بشوم عند الشوارع الم...
...,...,...,...
4162,ولا تكونوا كالذين نسوا الله ف أنساهم أنفسهم اس...,neu,__label__neu ولا تكونوا كالذين نسوا الله ف أنس...
4163,انشروا التذكير بالتكبير وأحيوا سنة هذه العشر ا...,neg,__label__neg انشروا التذكير بالتكبير وأحيوا سن...
4164,شوقي ولد الهدى فالكائنات ضياء وفم الزمان تبسم ...,neu,__label__neu شوقي ولد الهدى فالكائنات ضياء وفم...
4165,كلما رأيت أطفالا مشردين جائعين محاصرين لاجئين ...,neg,__label__neg كلما رأيت أطفالا مشردين جائعين مح...


In [11]:
model = fasttext.train_supervised(input=directory+'fasttext/text.txt')

In [18]:
model.predict("حمدا لله على سلامة أبنائنا المجند")

(('__label__pos',), array([0.54534626]))

In [ ]:
0.38748461+0.36599559+0.24654981

1.0000300100000001

In [14]:
model.predict("احمد جمال بودي جارد القهوه الزئبق الاحمر ناس تشبهلنا التبرع بالدم",k=3)

(('__label__neg', '__label__neu', '__label__pos'),
 array([0.33990178, 0.33336404, 0.3267642 ]))

In [16]:
model.test(directory+'fasttext/text.txt')

(4167, 0.5824334053275738, 0.5824334053275738)

In [19]:
data_test = pd.read_csv(directory+"araBERT/almost_cleaned_test.csv")
ides=[]
id= 0
labels = []
for tweet in data_test.tweet:
  id+=1
  ides.append(id)
  pred = model.predict(tweet)[0][0].replace("__label__",'')
  # print(line)
  labels.append(pred)
# text = '\n'.join(lines)

In [20]:
submission = pd.DataFrame({"Id":ides, "class":labels})
submission

,Id,class
0,1,neg
1,2,pos
2,3,pos
3,4,neu
4,5,neg
...,...,...
682,683,pos
683,684,neg
684,685,pos
685,686,neg


In [21]:
# submission = pd.DataFrame({"Id":ides, "class":labels})
# save (submission)
submission.to_csv("/content/drive/MyDrive/DEBI/uottawa/Ds applications/kaggle competition/fasttext/submission_fast.csv", index=False)